<a href="https://colab.research.google.com/github/garfield-gray/Optimization/blob/main/NonConvex/Cholesky_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

In [3]:
A = np.array([[6, 2, 1],
              [2, 8, 0],
              [1, 1, 9]])
np.linalg.cholesky(A)@np.linalg.cholesky(A).T

array([[6., 2., 1.],
       [2., 8., 1.],
       [1., 1., 9.]])